##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Обучение нескольких рабочих процессов с помощью Оценщика(Estimator)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/multi_worker_with_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

## Введение

Примечание: хотя вы можете использовать оценщики с `tf.distribute`, рекомендуется использовать Keras с `tf.distribute`, см. [Обучение нескольких воркеров с Keras](multi_worker_with_keras.ipynb). Обучение оценщика с помощью `tf.distribute.Strategy` имеет ограниченную поддержку.

В этом руководстве описывается, как можно использовать `tf.distribute.Strategy` для распределенного обучения нескольких воркеров с помощью `tf.estimator`. Если вы пишете код с использованием `tf.estimator` и хотите масштабироваться за пределы одной машины, это руководство для вас.

Перед тем, как начать, пожалуйста, прочтите руководство [распределенное обучение](../../guide/distribution_training.ipynb). 
[Учебное пособие по обучению работе с несколькими GPU](./keras.ipynb) также актуально, поскольку в этом учебном пособии используется такая же модель.


## Установка

Установите TensorFlow и требуемые импорты.

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

import os, json

## Функция ввода данных

В этом руководстве используется набор данных MNIST из [TensorFlow Datasets](https://www.tensorflow.org/datasets). Код здесь похож на [учебное пособие по работе с несколькими GPU](./keras.ipynb) с одним ключевым отличием: при использовании Оценщика для обучения нескольких воркеров необходимо разпределить датасет по количеству воркеров, чтобы гарантировать сходимость модели. Входные данные разделяются по количеству воркеров, таким образом каждый воркер обрабатывает отдельные части датасета - «1/num_workers».

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def input_fn(mode, input_context=None):
  datasets, info = tfds.load(name='mnist',
                                with_info=True,
                                as_supervised=True)
  mnist_dataset = (datasets['train'] if mode == tf.estimator.ModeKeys.TRAIN else
                   datasets['test'])

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

  if input_context:
    mnist_dataset = mnist_dataset.shard(input_context.num_input_pipelines,
                                        input_context.input_pipeline_id)
  return mnist_dataset.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Еще один разумный подход к достижению сходимости - это перемешать набор данных на основании рендомного числа для каждого воркера.

## Конфигурация для множества рабочих процессов

Одно из ключевых отличий этого руководства от [учебного пособия по работе с несколькими GPU](./keras.ipynb) - это настройка нескольких воркеров. 
Переменная среды `TF_CONFIG` - это стандартный способ указать конфигурацию кластера для каждого воркера, который является частью кластера.

`TF_CONFIG` состоит из двух компонентов: `cluster` и `task`. `cluster` содержит информацию обо всем кластере, а именно о параметрах воркеров и серверов в кластере. `task` содержит информацию о текущей задаче. Первый параметр `cluster` является одним и тем же для всех рабочих процессов и серверов в кластере, а второй компонент `task` отличается для каждого рабочего сервера и определяет свой `type` и `index`. В нашем примере тип задачи - worker, а индекс задачи - 0.

В целях иллюстрации в этом руководстве показано, как установить `TF_CONFIG` с двумя воркерами на localhost. На практике вы должны создать несколько воркеров на внешнем IP-адресе и порту и соответствующим образом указать это в  `TF_CONFIG` для каждого воркера, то есть изменить таск `index`.

Предупреждение: *Не выполняйте этот код в Colab.* Среда выполнения TensorFlow попытается создать сервер gRPC с указанным IP-адресом и портом, что, скорее всего, завершится ошибкой.

```
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': 0}
})
```


## Определение модели

Напишите слои, оптимизатор и функцию потерь для обучения. Здесь мы создадим модель, также как описано в [руководстве по работе с несколькими GPU](./keras.ipynb).

In [ ]:
LEARNING_RATE = 1e-4
def model_fn(features, labels, mode):
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  logits = model(features, training=False)

  if mode == tf.estimator.ModeKeys.PREDICT:
    predictions = {'logits': logits}
    return tf.estimator.EstimatorSpec(labels=labels, predictions=predictions)

  optimizer = tf.compat.v1.train.GradientDescentOptimizer(
      learning_rate=LEARNING_RATE)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(labels, logits)
  loss = tf.reduce_sum(loss) * (1. / BATCH_SIZE)
  if mode == tf.estimator.ModeKeys.EVAL:
    return tf.estimator.EstimatorSpec(mode, loss=loss)

  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=optimizer.minimize(
          loss, tf.compat.v1.train.get_or_create_global_step()))

Примечание. Хотя в этом примере скорость обучения фиксирована, в целом может потребоваться регулирование скорости обучения в зависимости от глобального размера пакета

## MultiWorkerMirroredStrategy

Для обучения модели используйте экземпляр `tf.distribute.experimental.MultiWorkerMirroredStrategy`. `MultiWorkerMirroredStrategy` создает копии всех переменных в слоях модели на каждом устройстве для всех рабочих процессов. Эта стратегия использует `CollectiveOps`, операцию TensorFlow для коллективного взаимодействия, чтобы агрегировать градиенты и синхронизировать переменные. В [руководстве по `tf.distribute.Strategy`](../../guide/distributed_training.ipynb) содержится более подробная информация об этой стратегии.

In [ ]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

## Обучение и оценка модели

Укажите распределенную стратегию в `RunConfig` для оценщика, а затем обучите и оцените модель, вызвав `tf.estimator.train_and_evaluate`. В этом руководстве обучение распределяется через указание стратегии в `train_distribute`. Также возможна распределенная оценка через указание `eval_distribute`.

In [ ]:
config = tf.estimator.RunConfig(train_distribute=strategy)

classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir='/tmp/multiworker', config=config)
tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(input_fn=input_fn),
    eval_spec=tf.estimator.EvalSpec(input_fn=input_fn)
)

## Оптимизация обучения

Теперь у вас есть модель и оценщик, который может работать с несколькими воркерами, на основе `tf.distribute.Strategy`. Вы можете попробовать следующие методы, чтобы оптимизировать производительность обучения нескольких воркеров:

* *Увеличьте размер пакета:* Размер пакета, указанный здесь, рассчитан на GPU. Как правило, рекомендуется использовать самый большой размер пакета, который умещается в памяти графического процессора.
* *Преобразование переменных:* Преобразуйте переменные в `tf.float`, если это возможно. Официальная модель ResNet включает [пример](https://github.com/tensorflow/models/blob/8367cf6dabe11adf7628541706b660821f397dce/official/resnet/resnet_model.py#L466) того, как это можно сделать.
* *Используйте коллективное взаимодействие:* `MultiWorkerMirroredStrategy` предоставляет несколько [реализаций коллективного взаимодействия](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/distribute/cross_device_ops.py).
    * `RING` реализует группы на основе кольца с использованием gRPC в качестве уровня связи между хостами.
    * `NCCL` использует [NCCL от Nvidia](https://developer.nvidia.com/nccl) для реализации групп.
    * `AUTO` переносит выбор в процесс выполнения.
    
Наилучший выбор коллективной реализации зависит от количества и типа графических процессоров, а также от сетевого соединения в кластере. Чтобы отменить автоматический выбор, укажите другое допустимое значение для параметра коммуникации конструктора `MultiWorkerMirroredStrategy`, например `communication = tf.distribute.experimental.CollectiveCommunication.NCCL`.

Посетите [Раздел производительности](../../guide/function.ipynb), чтобы узнать о других стратегиях и [инструментах](../../guide/profiler.md), которые можно использовать для оптимизации ваших моделей TensorFlow.


## Другие примеры кода

1. [Полный пример](https://github.com/tensorflow/ecosystem/tree/master/distribution_strategy) для обучения нескольких воркеров в тензорфлоу экосистеме с использованием шаблонов Kubernetes. Этот пример начинается с создания модели Keras и преобразует ее в оценщик с помощью API `tf.keras.estimator.model_to_estimator`.
2. [Официальные модели](https://github.com/tensorflow/models/tree/master/official), многие из которых можно настроить для работы с несколькими распределенными стратегиями.
